## A. Configuración General.

In [1]:
#1. Librerías.
%run "../librerias.ipynb"

/home/gmasi/.venv/lib/python3.12/site-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)
/home/gmasi/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#3. Constantes.
%run "../constantes.ipynb"

dataset_preprocesado = dataset_file_fe_all_3

cantidad_meses_train = "all"
ventana = 3

mes_train = mes_train_all_menos_3
mes_test = mes_test

In [3]:
#4. Funciones
%run "../funciones.ipynb"

In [4]:
#5. Lectura de datos.
files = [datasets_path + f"{mes}_{ventana}.parquet" for mes in mes_train + [mes_test]]

# Leer y combinar archivos
data = pd.concat([pd.read_parquet(file) for file in files], ignore_index=True)

In [5]:
#6. Pequeño pre-procesamiento sobre los datos.
#i. Cambio tipos de datos (Me lo toma como tipo de dato "object"...)
data['ctrx_quarter_normalizado'] = data['ctrx_quarter_normalizado'].astype(float)
#ii. Elimino columnas de último momento por Data Concept.
# columnas_de_interes_prestamos = data.filter(like='prestamos_personales').columns
# data.drop(columnas_de_interes_prestamos,axis=1,inplace=True)
#iii. Pesos y reclusterización.
data['clase_peso'] = 1.0
        
data.loc[data['clase_ternaria'] == 'BAJA+2', 'clase_peso'] = 1.00002
data.loc[data['clase_ternaria'] == 'BAJA+1', 'clase_peso'] = 1.00001

data['clase_binaria1'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)
data['clase_binaria2'] = np.where(data['clase_ternaria'] == 'CONTINUA', 0, 1)


In [6]:
#iv. Divido entre Train y Test.
train_data = data[data['foto_mes'].isin(mes_train)]
test_data = data[data['foto_mes'] == mes_test]

In [7]:
del data
import gc
gc.collect()

40

In [8]:
train_data

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,mactivos_margen,...,avg_historico_tc_proporcion_consumo_pesos,ratio_actual_tc_proporcion_adelanto_pesos,sumcum_tc_proporcion_adelanto_pesos,avg_historico_tc_proporcion_adelanto_pesos,ratio_actual_tc_proporcion_adelanto_dolares,sumcum_tc_proporcion_adelanto_dolares,avg_historico_tc_proporcion_adelanto_dolares,clase_peso,clase_binaria1,clase_binaria2
0,869143884,201904,1,0,56,104,5303.092950,65073.348317,1190.351318,1092.186603,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
1,869364346,201904,1,0,42,111,92.970157,18200.432998,0.000000,26.203259,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
2,869463381,201904,1,0,38,110,2412.401842,33321.770262,724.259510,-165.428020,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
3,869649487,201904,1,0,32,90,1603.061519,42416.606051,548.247359,-1185.422694,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
4,869743570,201904,1,0,43,110,2262.149729,19974.578045,1621.800927,-288.306770,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4193159,1147991305,202107,1,0,24,69,101.036840,464.823808,0.000000,85.603783,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
4193160,1148071168,202107,1,0,43,69,502.342905,16256.831511,277.437926,-266.359972,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
4193161,1148921495,202107,1,0,44,47,1868.898195,24788.191346,1515.024716,228.662323,...,1.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0
4193162,1149218456,202107,1,0,45,68,628.555502,10437.093974,0.000000,519.926343,...,0.0,NaN,0.0,0.0,NaN,0.0,0.0,1.0,0,0


In [9]:

#b. Datos para entrenar todo el modelo final para Kaggle.
X_train = train_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2', 'clase_binaria1'], axis=1)
y_train_binaria2 = train_data['clase_binaria2']
w_train = train_data['clase_peso']

#c. Datos de Test (a predecir).
X_test = test_data.drop(['clase_ternaria', 'clase_peso','clase_binaria2', 'clase_binaria1'], axis=1)
y_test_binaria2 = test_data['clase_binaria2']


In [10]:
del train_data
del test_data
gc.collect()


0

In [11]:
import random
# Semilla para el generador de números aleatorios
SEED = semillas[0]
random.seed(SEED)

# Obtener los meses únicos en el dataset
mes_column = X_train['foto_mes']
previous_months = sorted(mes_column.unique())
# Parámetros
BLOCK_SIZE = 3  # Cantidad de bloques
ELEMENTS_PER_BLOCK = 20  # Tamaño de cada bloque

# Generar bloques
month_blocks = []
for _ in range(BLOCK_SIZE):
    block = random.sample(previous_months, k=ELEMENTS_PER_BLOCK)
    block.sort()
    month_blocks.append(block)

# Imprimir los bloques generados
for i, block in enumerate(month_blocks, start=1):
    print(f"Bloque {i}: {block}")

Bloque 1: [201904, 201905, 201906, 201908, 201909, 201911, 202001, 202002, 202003, 202005, 202008, 202009, 202011, 202012, 202101, 202102, 202103, 202104, 202105, 202107]
Bloque 2: [201904, 201905, 201907, 201908, 201909, 201910, 201911, 201912, 202002, 202003, 202005, 202006, 202008, 202009, 202010, 202101, 202103, 202104, 202106, 202107]
Bloque 3: [201904, 201905, 201906, 201907, 201908, 201909, 201910, 201912, 202001, 202002, 202003, 202004, 202005, 202007, 202009, 202011, 202012, 202102, 202103, 202104]


In [ ]:
import optuna
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
import warnings

warnings.filterwarnings("ignore", message="Found `num_boost_round` in params. Will use it instead of argument")

def optimize_month(X_train_mes, y_train_mes, i):
    def objective(trial):
        learning_rate = trial.suggest_float('learning_rate', 0.001, 0.35)
        stopping_rounds = int(50 + 5 / learning_rate)
        # Hiperparámetros específicos a optimizar
        lgb_params = {
            'objective': 'binary',
            'metric': 'custom',
            'boosting_type': 'rf',
            'first_metric_only': True,
            'boost_from_average': True,
            'feature_pre_filter': False,
            'max_depth': -1,
            'num_threads': -1,
            'verbose': -1,
            'num_boost_round': trial.suggest_int('num_boost_round', 20, 500),
            'max_bin': trial.suggest_int('max_bin', 20, 200),
            'num_leaves': trial.suggest_int('num_leaves', 8, 300),
            'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 1000),
            'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
            'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
            'learning_rate': learning_rate,
        }
        N_FOLDS = 5
        N_ITER = 1  # Cantidad de iteraciones con diferentes semillas
        scores_iter = []
    
        for i in range(N_ITER):  # Cambiamos la semilla en cada iteración
            skf_meta = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=i)
            scores_meta = []
    
            for train_idx, val_idx in skf_meta.split(X_train_mes, y_train_mes):
                meta_X_train, meta_X_val = X_train_mes.iloc[train_idx], X_train_mes.iloc[val_idx]
                meta_y_train, meta_y_val = y_train_mes.iloc[train_idx], y_train_mes.iloc[val_idx]
    
                model = lgb.LGBMClassifier(**lgb_params, random_state=i, stopping_rounds=stopping_rounds)
                model.fit(meta_X_train, meta_y_train)
                
                val_pred = model.predict(meta_X_val)
                max_ganancia = ganancia_prob(val_pred, meta_y_val)
                scores_meta.append(max_ganancia)
            
            # Promedio de ganancias para los K-Folds en la iteración actual
            iter_avg_score = np.mean(scores_meta) * N_FOLDS
            scores_iter.append(iter_avg_score)

        # Promedio de ganancias para las N iteraciones con distintas semillas
        return np.mean(scores_iter)

    # Optimizar los hiperparámetros para este mes
    storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
    study_name = "exp_122_lgbm_stack_block"  + str(i)
    study = optuna.create_study(
        direction="maximize",
        study_name=study_name,
        storage=storage_name,
        load_if_exists=True,
    )
    study.optimize(objective, n_trials=10)
    return study.best_params

best_params_per_month = {}

for i, block in enumerate(month_blocks):
    if i != 2:
        continue
    print(f"Optimizando para el bloque {i}...")
    X_train_mes = X_train[X_train['foto_mes'].isin(block)]
    y_train_mes = y_train_binaria2[X_train['foto_mes'].isin(block)]
    best_params_per_month[i] = optimize_month(X_train_mes, y_train_mes, i)



Optimizando para el bloque 2...


[I 2024-11-29 17:10:01,176] A new study created in RDB with name: exp_122_lgbm_stack_block2
[I 2024-11-29 17:26:21,842] Trial 0 finished with value: 3169068000.0 and parameters: {'learning_rate': 0.06485485089217778, 'num_boost_round': 382, 'max_bin': 145, 'num_leaves': 61, 'min_data_in_leaf': 963, 'feature_fraction': 0.3582440884009176, 'bagging_fraction': 0.9531439791775027}. Best is trial 0 with value: 3169068000.0.


In [ ]:

lgb_models = {}
for i, block in enumerate(month_blocks):
    params = best_params_per_month[i]
    X_train_mes = X_train[X_train['foto_mes'].isin(block)]
    y_train_mes = y_train_binaria2[X_train['foto_mes'].isin(block)]
    stopping_rounds = int(50 + 5 / params['learning_rate'])
    params.update({
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'rf',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_depth': -1,
        'num_threads': -1,
        'verbose': -1,
    })
    model = lgb.LGBMClassifier(**params, random_state=i, stopping_rounds=stopping_rounds)
    model.fit(X_train_mes, y_train_mes)
    lgb_models[i] = model


In [ ]:
oof_predictions = np.zeros((X_validation.shape[0], len(month_blocks)))

for i, block in enumerate(month_blocks):
    model = lgb_models[i]
    oof_predictions[:, i] = model.predict_proba(X_validation)[:, 1]

In [ ]:
def objective_meta(trial):
    # Hiperparámetros específicos a optimizar

    learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
    stopping_rounds = int(50 + 5 / learning_rate)
    meta_params = {
        'objective': 'binary',
        'metric': 'custom',
        'boosting_type': 'rf',
        'first_metric_only': True,
        'boost_from_average': True,
        'feature_pre_filter': False,
        'max_depth': -1,
        'num_threads': -1,
        'verbose': -1,
        'num_boost_round': trial.suggest_int('num_boost_round', 20, 500),
        'max_bin': trial.suggest_int('max_bin', 20, 200),
        'num_leaves': trial.suggest_int('num_leaves', 8, 300),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 10, 1000),
        'feature_fraction': trial.suggest_float('feature_fraction', 0.1, 1.0),
        'bagging_fraction': trial.suggest_float('bagging_fraction', 0.1, 1.0),
        'learning_rate': learning_rate,
    }
    N_FOLDS = 5
    N_ITER = 1  # Cantidad de iteraciones con diferentes semillas
    scores_iter = []

    for i in range(N_ITER):  # Cambiamos la semilla en cada iteración
        skf_meta = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=i)
        scores_meta = []

        for train_idx, val_idx in skf_meta.split(meta_X, meta_y):
            meta_X_train, meta_X_val = meta_X.iloc[train_idx], meta_X.iloc[val_idx]
            meta_y_train, meta_y_val = meta_y.iloc[train_idx], meta_y.iloc[val_idx]

            # Configura el modelo con la semilla actual
            meta_model = lgb.LGBMClassifier(**meta_params, random_state=i, stopping_rounds=stopping_rounds)
            meta_model.fit(meta_X_train, meta_y_train)
            
            val_pred = meta_model.predict(meta_X_val)
            max_ganancia = ganancia_prob(val_pred, meta_y_val)
            scores_meta.append(max_ganancia)
        
        # Promedio de ganancias para los K-Folds en la iteración actual
        iter_avg_score = np.mean(scores_meta) * N_FOLDS
        scores_iter.append(iter_avg_score)

    # Promedio de ganancias para las N iteraciones con distintas semillas
    return np.mean(scores_iter)

meta_features = pd.DataFrame(oof_predictions, columns=[f'model_mes_{i}' for i, block in enumerate(month_blocks)])
meta_features = pd.concat([meta_features, X_validation.reset_index(drop=True)], axis=1)
meta_X = meta_features
meta_y = y_validation_binaria2

# no-name-3a3c49eb-5af8-4d2d-82a4-33825140e7a6
# [I 2024-11-16 13:21:13,038] Trial 129 finished with value: 309176000.0 and parameters: {'learning_rate': 0.2804332312296757, 'num_boost_round': 210, 'max_bin': 20, 'num_leaves': 86, 'min_data_in_leaf': 94, 'feature_fraction': 0.4499833274696736, 'bagging_fraction': 0.7410741485338036}. Best is trial 129 with value: 309176000.0.

storage_name = "sqlite:///" + db_path + "optimization_lgbm.db"
study_name = "exp_121_lgbm_stack_blocks"
study = optuna.create_study(
    direction="maximize",
    study_name=study_name,
    storage=storage_name,
    load_if_exists=True,
)
study.optimize(objective_meta, n_trials=50)


In [ ]:
meta_params = study.best_params
stopping_rounds = int(50 + 5 / meta_params['learning_rate'])
meta_params.update({
    'objective': 'binary',
    'metric': 'custom',
    'boosting_type': 'rf',
    'first_metric_only': True,
    'boost_from_average': True,
    'feature_pre_filter': False,
    'max_depth': -1,
    'num_threads': -1,
    'verbose': -1,
})
meta_model = lgb.LGBMClassifier(**meta_params, random_state=i, stopping_rounds=stopping_rounds)
meta_model.fit(meta_X, meta_y)

In [40]:
oof_predictions = np.zeros((X_test.shape[0], len(month_blocks)))

# Entrenar modelos mensuales y generar predicciones sobre el último mes
for i, block in enumerate(month_blocks):
    # Guardar las probabilidades predichas para el meta-modelo usando el último mes
    oof_predictions[:, i] = lgb_models[i].predict_proba(X_test)[:, 1]
    
# Crear dataset para el meta-modelo
meta_features = pd.DataFrame(oof_predictions, columns=[f'model_mes_{i}' for i, block in enumerate(month_blocks)])
meta_features = pd.concat([meta_features, X_test.reset_index(drop=True)], axis=1)

predicciones = meta_model.predict(meta_features)

ValueError: Number of features of the model must match the input. Model n_features_ is 915 and input n_features is 916

In [10]:
#4. Visualizo los resultados del estudio, para modificar los rangos de análisis.

In [ ]:
optuna.visualization.plot_optimization_history(study)

In [ ]:
plot_param_importances(study)

In [ ]:
plot_slice(study)

In [ ]:
plot_contour(study)

In [ ]:
plot_contour(study, params=['num_leaves','min_data_in_leaf'] )

In [20]:
study.best_trial.params

{'learning_rate': 0.07861021583599366,
 'num_boost_round': 263,
 'max_bin': 188,
 'num_leaves': 94,
 'min_data_in_leaf': 310,
 'feature_fraction': 0.9954418131347904,
 'bagging_fraction': 0.5850033373679738}

In [23]:
lgb_models

{0: LGBMClassifier(bagging_fraction=0.6139544534868907, boost_from_average=True,
                boosting_type='rf', feature_fraction=0.5959985554142665,
                feature_pre_filter=False, first_metric_only=True,
                learning_rate=0.25995095782781696, max_bin=31, metric='custom',
                min_data_in_leaf=316, num_boost_round=81, num_leaves=295,
                num_threads=-1, objective='binary', random_state=0, verbose=-1),
 1: LGBMClassifier(bagging_fraction=0.6986367064998396, boost_from_average=True,
                boosting_type='rf', feature_fraction=0.5513776690595713,
                feature_pre_filter=False, first_metric_only=True,
                learning_rate=0.2326848732220626, max_bin=200, metric='custom',
                min_data_in_leaf=692, num_boost_round=277, num_leaves=298,
                num_threads=-1, objective='binary', random_state=1, verbose=-1),
 2: LGBMClassifier(bagging_fraction=0.9809207286219903, boost_from_average=True,
        

In [19]:
#ii. Dataframe.

for i in lgb_models:
    # Graficar feature importace de cada mes
    lgb_model = lgb_models[i]
    importance = lgb_model.booster_.feature_importance()
    features = lgb_model.booster_.feature_name()
    importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})
    importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)
    print(importance_df.head(10))
    
#a. Extract feature importance and feature names.
importance = meta_model.booster_.feature_importance()
features = meta_model.booster_.feature_name()

#b. Create a dataframe for better visualization.
importance_df = pd.DataFrame({'Feature': features, 'Importance': importance})

#c. Sort by importance in descending order.
importance_df = importance_df.sort_values(by='Importance', ascending=False).reset_index(drop=True)

#d. show.
importance_df.head(50)

                                        Feature  Importance
0                                      foto_mes        1609
1                     ratio_actual_ctrx_quarter         726
2            ratio_actual_mprestamos_personales         544
3                                mcuentas_saldo         496
4                             tc_fechalta_menor         425
5                         mprestamos_personales         392
6            avg_historico_mrentabilidad_annual         390
7  ratio_actual_m_promedio_prestamos_personales         368
8                        m_promedio_caja_ahorro         366
9                   sumcum_mrentabilidad_annual         355
                                        Feature  Importance
0                                      foto_mes        6826
1                     ratio_actual_ctrx_quarter        2884
2                                mcuentas_saldo        2669
3            ratio_actual_mprestamos_personales        2419
4                   sumcum_mrentabilidad

,Feature,Importance
0,model_mes_0,829
1,model_mes_2,738
2,model_mes_1,679
3,mrentabilidad_annual_slope_1,350
4,m_promedio_transferencias_recibidas,320
5,ratio_actual_mcuenta_corriente,294
6,mcuentas_saldo_slope_1,290
7,ratio_actual_mpasivos_margen,275
8,avg_historico_mrentabilidad_annual,257
9,mrentabilidad_annual,250


In [41]:
X_test.drop(['Probabilidad'], axis=1, inplace=True)


In [42]:
oof_predictions = np.zeros((X_test.shape[0], len(month_blocks)))

# Entrenar modelos mensuales y generar predicciones sobre el último mes
for i, block in enumerate(month_blocks):
    # Guardar las probabilidades predichas para el meta-modelo usando el último mes
    oof_predictions[:, i] = lgb_models[i].predict_proba(X_test)[:, 1]
    
# Crear dataset para el meta-modelo
meta_features = pd.DataFrame(oof_predictions, columns=[f'model_mes_{i}' for i, block in enumerate(month_blocks)])
meta_features = pd.concat([meta_features, X_test.reset_index(drop=True)], axis=1)

predicciones = meta_model.predict(meta_features)

In [43]:
predicciones.sum()

8162

In [46]:
X_test['Probabilidad'] = predicciones

In [22]:
#iii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
#iv. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(9000,14000,200)
#v. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1
i = 0
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_OH_6_6_00{}.csv".format(num_subida_kaggle) #-X meses, con df de -X meses, número de intento.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    i += 1

Cantidad de clientes 9000
Cantidad de clientes 9200
Cantidad de clientes 9400
Cantidad de clientes 9600
Cantidad de clientes 9800
Cantidad de clientes 10000
Cantidad de clientes 10200
Cantidad de clientes 10400
Cantidad de clientes 10600
Cantidad de clientes 10800
Cantidad de clientes 11000
Cantidad de clientes 11200
Cantidad de clientes 11400
Cantidad de clientes 11600
Cantidad de clientes 11800
Cantidad de clientes 12000
Cantidad de clientes 12200
Cantidad de clientes 12400
Cantidad de clientes 12600
Cantidad de clientes 12800
Cantidad de clientes 13000
Cantidad de clientes 13200
Cantidad de clientes 13400
Cantidad de clientes 13600
Cantidad de clientes 13800


In [ ]:
#7. Guardamos el modelo.
# Primer dígito cuantos meses para atrás desde 06/21, segundo dígito número data drifting, tercer dígito número de entrenamiento.
model_lgb.save_model(modelos_path + 'lgbm_{}_{}_undersampling.txt'.format(cantidad_meses_train,ventana))

In [ ]:
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################
############## Primera subida orientativa a Kaggle previo medir semillas, etc. #####################
####################################################################################################
####################################################################################################
####################################################################################################
####################################################################################################

In [ ]:
#8. Volvemos a leer el modelo.
model_lgb = lgb.Booster(model_file= modelos_path + 'lgbm_{}_{}_undersampling.txt'.format(cantidad_meses_train,ventana))

In [44]:
#a. Importo librería.
from kaggle.api.kaggle_api_extended import KaggleApi
#b. Configura el API de Kaggle
api = KaggleApi()
api.authenticate()

In [47]:
#iii. Ordenamos a los clientes por probabilidad de ser "BAJA" de forma descendente.
tb_entrega = X_test.sort_values(by='Probabilidad', ascending=False)
#iv. Genero una lista de distintos cortes candidatos, para enviar a Kaggle.
cortes = range(9000,14000,200)
#v. Generamos las distintas predicciones de clases a partir de los distintos cortes posibles.
num_subida_kaggle = 1

i = 0
for envios in cortes:
    #1. Le ponemos clase 1 ("BAJA") a los primeros "envios" con mayor probabilidad.
    tb_entrega['Predicted'] = 0
    tb_entrega.iloc[:envios, tb_entrega.columns.get_loc('Predicted')] = 1
    resultados = tb_entrega[["numero_de_cliente", 'Predicted']].reset_index(drop=True)
    
    print("Cantidad de clientes {}".format(envios))
    #2. Guardamos el archivo para Kaggle.
    nombre_archivo = "K_OH_6_6_00{}.csv".format(num_subida_kaggle) #-X meses, con df de -X meses, número de intento.
    ruta_archivo= "{}/{}".format(exp_path,nombre_archivo)
    resultados.to_csv(ruta_archivo, index=False)
    
    num_subida_kaggle += 1
    i += 1
    
    #3. Envío a Kaggle.
    #a. Defino los parámetros claves.
    mensaje = f'DF {ventana}, punto_corte: {envios}.'
    competencia = 'dm-ey-f-2024-segunda'
    #c. Subo la Submission.
    while i<=15:
        print(i)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
        break
    else:
        print("Esperamos 30 segundos...")
        time.sleep(30)
        api.competition_submit(file_name=ruta_archivo, message=mensaje, competition=competencia)
        print("Submission successful!")
        i= 0

Cantidad de clientes 9000
1


100%|██████████| 1.93M/1.93M [00:00<00:00, 2.94MB/s]


Submission successful!
Cantidad de clientes 9200
2


100%|██████████| 1.93M/1.93M [00:00<00:00, 3.30MB/s]


Submission successful!
Cantidad de clientes 9400
3


...

Submission successful!
Cantidad de clientes 13600
8


100%|██████████| 1.93M/1.93M [00:00<00:00, 3.17MB/s]


Submission successful!
Cantidad de clientes 13800
9


100%|██████████| 1.93M/1.93M [00:00<00:00, 3.16MB/s]


Submission successful!
